In [19]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn as nn
import torch.nn.functional as F
import math
import torch.optim as optim
import numpy as np
from torchvision.models import resnet18, ResNet18_Weights
from torchvision import transforms
from PIL import Image
import os
import matplotlib.pyplot as plt
from torch.nn.utils.rnn import pad_sequence
from torchmetrics import MeanAbsoluteError
from sklearn.preprocessing import StandardScaler
from torchvision.transforms import Resize, ToTensor
from torchvision.transforms.functional import to_pil_image
# from models.resnet import ResNet18VisualEncoder


In [20]:
# Attempt to load the dataset with allow_pickle=True
trajectories = np.load('trajectories_2.npy', allow_pickle=True)

# Display the shape and a few sample data points of the loaded trajectories
trajectories.shape, trajectories

((10,),
 array([{'map_name': 'maze_1.png', 'start': (71, 69), 'goal': (22, 36), 'trajectory': [(71, 68), (72, 68), (73, 68), (74, 68), (75, 68), (76, 68), (76, 67), (76, 66), (76, 65), (76, 64), (76, 63), (76, 62), (76, 61), (76, 60), (77, 60), (78, 60), (79, 60), (80, 60), (81, 60), (82, 60), (83, 60), (84, 60), (84, 59), (84, 58), (84, 57), (84, 56), (84, 55), (84, 54), (84, 53), (84, 52), (85, 52), (86, 52), (87, 52), (88, 52), (89, 52), (90, 52), (91, 52), (92, 52), (92, 51), (92, 50), (92, 49), (92, 48), (92, 47), (92, 46), (92, 45), (92, 44), (92, 43), (92, 42), (92, 41), (92, 40), (92, 39), (91, 39), (90, 39), (89, 39), (88, 39), (87, 39), (86, 39), (85, 39), (84, 39), (83, 39), (82, 39), (81, 39), (80, 39), (79, 39), (78, 39), (77, 39), (76, 39), (75, 39), (74, 39), (73, 39), (72, 39), (71, 39), (70, 39), (69, 39), (68, 39), (68, 38), (68, 37), (68, 36), (68, 35), (68, 34), (68, 33), (68, 32), (68, 31), (67, 31), (66, 31), (65, 31), (64, 31), (63, 31), (62, 31), (61, 31), (60, 

In [21]:

def cosine_beta_schedule(timesteps=1000, s=0.008):
    """
    Cosine scheduler for beta values.
    Args:
        timesteps (int): Number of timesteps.
        s (float): Small constant to avoid beta being zero at t=0.
    Returns:
        np.ndarray: Array of beta values.
    """
    steps = np.arange(timesteps + 1) / timesteps
    alphas_cumprod = np.cos(((steps + s) / (1 + s)) * np.pi / 2) ** 2
    betas = 1 - (alphas_cumprod[1:] / alphas_cumprod[:-1])
    return np.clip(betas, 0, 0.999)


In [22]:

def add_noise(trajectory, betas, noise_scale=1.0):
    """
    Add noise to the trajectory based on beta values.
    Args:
        trajectory (list of tuple): The original trajectory.
        betas (np.ndarray): Array of beta values.
    Returns:
        list of np.ndarray: Noisy trajectories.
    """
    noisy_trajectories = []
    trajectory = np.array(trajectory)
    
    for beta in betas:
        noise = np.random.normal(scale=np.sqrt(beta * noise_scale), size=trajectory.shape)
        noisy_trajectory = trajectory + noise
        noisy_trajectories.append(noisy_trajectory)
    
    return noisy_trajectories


In [23]:

def forward_diffusion_process(trajectories, timesteps=100,noise_scale=1.0):
    """
    Forward diffusion process that adds noise to trajectories.
    Args:
        trajectories (list of dict): List of trajectories with metadata.
        timesteps (int): Number of diffusion steps.
    Returns:
        list of dict: Noisy trajectories with metadata.
    """
    
    betas = cosine_beta_schedule(timesteps)
    noisy_trajectories = []
    # noise_scale = noise_scale
    noise_scale = 4.0


    for trajectory in trajectories:
        noisy_traj = add_noise(trajectory['trajectory'], betas, noise_scale)
        noisy_trajectories.append({
            'map_name': trajectory['map_name'],
            # 'start': trajectory['start'],
            # 'goal': trajectory['goal'],
            'noisy_trajectories': noisy_traj
        })
    
    return noisy_trajectories

# Apply forward diffusion process to the loaded trajectories
timesteps = 1000
noisy_trajectories = forward_diffusion_process(trajectories, timesteps)

# Display a sample of noisy trajectories
noisy_trajectories[0]['noisy_trajectories'][:5]  # Display first 5 steps of noisy trajectories for the first sample

[array([[70.98808733, 67.98594423],
        [72.00365262, 67.98375729],
        [73.00784788, 68.00256224],
        [73.99617354, 67.98941091],
        [74.97957531, 68.01567585],
        [76.00081771, 68.01938333],
        [75.98615097, 66.99170123],
        [76.00962671, 66.02117767],
        [76.01568043, 65.00453789],
        [75.96317917, 64.01515043],
        [75.9843247 , 62.99495968],
        [76.01975032, 61.97541425],
        [76.00652603, 60.99149027],
        [75.9853027 , 60.00454893],
        [76.97262007, 59.98910927],
        [77.96608617, 59.99104978],
        [79.00159502, 60.01029041],
        [79.99488671, 60.01371621],
        [81.00427554, 59.99608319],
        [82.01266178, 60.01679734],
        [82.97755816, 60.00738054],
        [84.02460967, 60.01200285],
        [84.0039159 , 58.9991132 ],
        [84.00589033, 57.99427501],
        [83.99989228, 57.01580525],
        [84.01998128, 55.99428637],
        [83.98774647, 54.98152088],
        [84.01108704, 54.004

In [24]:
# Prepare the data to be saved in the noisy_trajectories_2.npy file
output_data = []

for noisy_traj in noisy_trajectories:
    output_data.append({
        'map_name': noisy_traj['map_name'],
        # 'start': noisy_traj['start'],
        # 'goal': noisy_traj['goal'],
        'noisy_trajectory': noisy_traj['noisy_trajectories'][-1] # Save the final noisy trajectory
    })

# Save the data to a .npy file
output_file_path = 'noisy_trajectories_2.npy'
np.save(output_file_path, output_data)

output_file_path


'noisy_trajectories_2.npy'